In [6]:
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import pandas as pd

csv_file = "all_stocks_5yr.csv"

if not os.path.exists(csv_file):
    raise FileNotFoundError(f"Error: {csv_file} not found. Please place the CSV file in the same folder as this script.")

df = pd.read_csv(csv_file)
print("Dataset loaded successfully.")

Dataset loaded successfully.


In [2]:
import pandas as pd
df = pd.read_csv(csv_file)

In [3]:
filtered_df = df[df['close'] > 100]
print(filtered_df.head())

            date    open    high     low   close    volume  name
1643  2014-08-19   99.41  100.68   99.32  100.53  69399270  AAPL
1644  2014-08-20  100.44  101.09   99.95  100.57  52699192  AAPL
1645  2014-08-21  100.57  100.94  100.11  100.58  33478198  AAPL
1646  2014-08-22  100.29  101.47  100.19  101.32  44183834  AAPL
1647  2014-08-25  101.79  102.17  101.28  101.54  40270173  AAPL


In [7]:
!pip install polars

  Using cached polars-1.24.0-cp39-abi3-win_amd64.whl.metadata (15 kB)
Using cached polars-1.24.0-cp39-abi3-win_amd64.whl (34.7 MB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import polars as pl
df_polars = pl.read_csv(csv_file)

filtered_df_polars = df_polars.filter(pl.col("close") > 100)
print(filtered_df_polars.head())

shape: (5, 7)
┌─────┬─────┬─────┬─────┬─────┬────────┬──────┐
│ dat ┆ ope ┆ hig ┆ low ┆ clo ┆ volume ┆ name │
│ e   ┆ n   ┆ h   ┆ --- ┆ se  ┆ ---    ┆ ---  │
│ --- ┆ --- ┆ --- ┆ f64 ┆ --- ┆ i64    ┆ str  │
│ str ┆ f64 ┆ f64 ┆     ┆ f64 ┆        ┆      │
╞═════╪═════╪═════╪═════╪═════╪════════╪══════╡
│ 201 ┆ 99. ┆ 100 ┆ 99. ┆ 100 ┆ 693992 ┆ AAPL │
│ 4-0 ┆ 41  ┆ .68 ┆ 32  ┆ .53 ┆ 70     ┆      │
│ 8-1 ┆     ┆     ┆     ┆     ┆        ┆      │
│ 9   ┆     ┆     ┆     ┆     ┆        ┆      │
│ 201 ┆ 100 ┆ 101 ┆ 99. ┆ 100 ┆ 526991 ┆ AAPL │
│ 4-0 ┆ .44 ┆ .09 ┆ 95  ┆ .57 ┆ 92     ┆      │
│ 8-2 ┆     ┆     ┆     ┆     ┆        ┆      │
│ 0   ┆     ┆     ┆     ┆     ┆        ┆      │
│ 201 ┆ 100 ┆ 100 ┆ 100 ┆ 100 ┆ 334781 ┆ AAPL │
│ 4-0 ┆ .57 ┆ .94 ┆ .11 ┆ .58 ┆ 98     ┆      │
│ 8-2 ┆     ┆     ┆     ┆     ┆        ┆      │
│ 1   ┆     ┆     ┆     ┆     ┆        ┆      │
│ 201 ┆ 100 ┆ 101 ┆ 100 ┆ 101 ┆ 441838 ┆ AAPL │
│ 4-0 ┆ .29 ┆ .47 ┆ .19 ┆ .32 ┆ 34     ┆      │
│ 8-2 ┆     ┆     ┆     ┆ 

In [9]:
%timeit pd.read_csv(csv_file)

%timeit pl.read_csv(csv_file)

329 ms ± 29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
20.5 ms ± 125 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
!pip install ta

  Using cached ta-0.11.0-py3-none-any.whl



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import os
import pandas as pd
import ta

script_dir = os.getcwd()

csv_file = os.path.join(script_dir, "all_stocks_5yr.csv")

if not os.path.exists(csv_file):
    print(f"Error: {csv_file} not found. Please place the CSV file in the same folder as this script.")
else:
    df = pd.read_csv(csv_file).sort_values(by="date")

    df["SMA_20"] = df["close"].rolling(window=20).mean()
    df["EMA_20"] = df["close"].ewm(span=20, adjust=False).mean()

    macd = ta.trend.MACD(df["close"])
    df["MACD"] = macd.macd()
    df["MACD_Signal"] = macd.macd_signal()

    bb = ta.volatility.BollingerBands(df["close"], window=20)
    df["BB_High"] = bb.bollinger_hband()
    df["BB_Low"] = bb.bollinger_lband()

    df["Lowest_Low"] = df["low"].rolling(window=14).min()
    df["Highest_High"] = df["high"].rolling(window=14).max()
    df["Stoch"] = 100 * ((df["close"] - df["Lowest_Low"]) / (df["Highest_High"] - df["Lowest_Low"]))

    df.drop(columns=["Lowest_Low", "Highest_High"], inplace=True)

    df.ffill(inplace=True)
    df.bfill(inplace=True)

    enhanced_csv_pandas = os.path.join(script_dir, "all_stocks_5yr_enhanced_pandas.csv")
    df.to_csv(enhanced_csv_pandas, index=False)

    print("Checking for null values in Pandas-enhanced dataset (AFTER Forward Fill):")
    print(df.isnull().sum())

    print(f"\nPandas-enhanced dataset saved: {enhanced_csv_pandas}")



Checking for null values in Pandas-enhanced dataset (AFTER Forward Fill):
date           0
open           0
high           0
low            0
close          0
volume         0
name           0
SMA_20         0
EMA_20         0
MACD           0
MACD_Signal    0
BB_High        0
BB_Low         0
Stoch          0
dtype: int64

Pandas-enhanced dataset saved: C:\Users\katig\OneDrive\Desktop\Katigbak_300366535_Assignment1-main\all_stocks_5yr_enhanced_pandas.csv


In [22]:
import os
import polars as pl

script_dir = os.getcwd()

csv_file = os.path.join(script_dir, "all_stocks_5yr.csv")
enhanced_csv_polars = os.path.join(script_dir, "all_stocks_5yr_enhanced_polars.csv")

if not os.path.exists(csv_file):
    print(f"Error: {csv_file} not found. Please place the CSV file in the same folder as this script.")
else:
    df_polars = pl.read_csv(csv_file).sort("date")

    df_polars = df_polars.with_columns([
        df_polars["close"].rolling_mean(window_size=20).alias("SMA_20"),
        df_polars["close"].ewm_mean(span=20).alias("EMA_20"),
    ])

    def compute_macd(df, column="close"):
        short_ema = df[column].ewm_mean(span=12)
        long_ema = df[column].ewm_mean(span=26)
        macd = short_ema - long_ema
        macd_signal = macd.ewm_mean(span=9)
        return macd, macd_signal

    macd, macd_signal = compute_macd(df_polars, "close")
    df_polars = df_polars.with_columns(macd.alias("MACD"), macd_signal.alias("MACD_Signal"))

    def compute_bollinger_bands(df, column="close", window=20):
        sma = df[column].rolling_mean(window_size=window)
        std_dev = df[column].rolling_std(window_size=window)
        upper_band = sma + (2 * std_dev)
        lower_band = sma - (2 * std_dev)
        return upper_band, lower_band

    bb_high, bb_low = compute_bollinger_bands(df_polars, "close", 20)
    df_polars = df_polars.with_columns(bb_high.alias("BB_High"), bb_low.alias("BB_Low"))

    df_polars = df_polars.with_columns([
        df_polars["low"].rolling_min(window_size=14).alias("Lowest_Low"),
        df_polars["high"].rolling_max(window_size=14).alias("Highest_High"),
    ])
    df_polars = df_polars.with_columns(
        ((df_polars["close"] - df_polars["Lowest_Low"]) / (df_polars["Highest_High"] - df_polars["Lowest_Low"]) * 100).alias("Stoch")
    )

    df_polars = df_polars.drop(["Lowest_Low", "Highest_High"])

    df_polars = df_polars.fill_null(strategy="forward").fill_null(strategy="backward")

    df_polars.write_csv(enhanced_csv_polars)

    print("Checking for null values in Polars-enhanced dataset (AFTER Forward Fill):")
    print(df_polars.null_count())

    print(f"\nPolars-enhanced dataset saved: {enhanced_csv_polars}")



Checking for null values in Polars-enhanced dataset (AFTER Forward Fill):
shape: (1, 14)
┌─────┬─────┬─────┬─────┬───┬─────┬─────┬─────┬─────┐
│ dat ┆ ope ┆ hig ┆ low ┆ … ┆ MAC ┆ BB_ ┆ BB_ ┆ Sto │
│ e   ┆ n   ┆ h   ┆ --- ┆   ┆ D_S ┆ Hig ┆ Low ┆ ch  │
│ --- ┆ --- ┆ --- ┆ u32 ┆   ┆ ign ┆ h   ┆ --- ┆ --- │
│ u32 ┆ u32 ┆ u32 ┆     ┆   ┆ al  ┆ --- ┆ u32 ┆ u32 │
│     ┆     ┆     ┆     ┆   ┆ --- ┆ u32 ┆     ┆     │
│     ┆     ┆     ┆     ┆   ┆ u32 ┆     ┆     ┆     │
╞═════╪═════╪═════╪═════╪═══╪═════╪═════╪═════╪═════╡
│ 0   ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ 0   ┆ 0   ┆ 0   ┆ 0   │
└─────┴─────┴─────┴─────┴───┴─────┴─────┴─────┴─────┘

Polars-enhanced dataset saved: C:\Users\katig\OneDrive\Desktop\Katigbak_300366535_Assignment1-main\all_stocks_5yr_enhanced_polars.csv


In [13]:
!pip install scikit-learn

  Using cached scikit_learn-1.6.1-cp313-cp313-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.2-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp313-cp313-win_amd64.whl (11.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp313-cp313-win_amd64.whl (41.0 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import os
import pandas as pd
import polars as pl
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

script_dir = os.getcwd()

csv_pandas = os.path.join(script_dir, "all_stocks_5yr_enhanced_pandas.csv")
csv_polars = os.path.join(script_dir, "all_stocks_5yr_enhanced_polars.csv")

if not os.path.exists(csv_pandas):
    print(f"Error: {csv_pandas} not found. Please place the enhanced Pandas CSV in the same folder as this script.")
else:
    df_pandas = pd.read_csv(csv_pandas)

if not os.path.exists(csv_polars):
    print(f"Error: {csv_polars} not found. Please place the enhanced Polars CSV in the same folder as this script.")
else:
    df_polars = pl.read_csv(csv_polars)

if 'df_pandas' in locals():
    print("Pandas-enhanced dataset loaded successfully!")

if 'df_polars' in locals():
    print("Polars-enhanced dataset loaded successfully!")



Pandas-enhanced dataset loaded successfully!
Polars-enhanced dataset loaded successfully!


In [28]:
df_pandas = df_pandas.sort_values(by=["date"])
df_pandas["target"] = df_pandas["close"].shift(-1)
df_pandas.dropna(inplace=True)


In [29]:
df_polars = df_polars.sort("date")
df_polars = df_polars.with_columns(df_polars["close"].shift(-1).alias("target"))
df_polars = df_polars.drop_nulls()


In [30]:
features = ["SMA_20", "EMA_20", "MACD", "MACD_Signal", "BB_High", "BB_Low", "Stoch"]
X_pandas = df_pandas[features]
y_pandas = df_pandas["target"]

X_polars = df_polars.select(features)
y_polars = df_polars.select("target")


In [31]:
X_train_pandas, X_test_pandas, y_train_pandas, y_test_pandas = train_test_split(
    X_pandas, y_pandas, test_size=0.2, random_state=42, shuffle=False
)


In [32]:
X_train_polars, X_test_polars, y_train_polars, y_test_polars = train_test_split(
    X_polars.to_pandas(), y_polars.to_pandas(), test_size=0.2, random_state=42, shuffle=False
)


In [33]:
scaler = StandardScaler()

X_train_pandas_scaled = scaler.fit_transform(X_train_pandas)
X_test_pandas_scaled = scaler.transform(X_test_pandas)

X_train_polars_scaled = scaler.fit_transform(X_train_polars)
X_test_polars_scaled = scaler.transform(X_test_polars)



In [34]:
lr_model_pandas = LinearRegression()
lr_model_pandas.fit(X_train_pandas_scaled, y_train_pandas)

lr_model_polars = LinearRegression()
lr_model_polars.fit(X_train_polars_scaled, y_train_polars)


LinearRegression()

In [35]:
y_pred_lr_pandas = lr_model_pandas.predict(X_test_pandas_scaled)
y_pred_lr_polars = lr_model_polars.predict(X_test_polars_scaled)


In [36]:
print("Linear Regression - Pandas")
print("MAE:", mean_absolute_error(y_test_pandas, y_pred_lr_pandas))
print("MSE:", mean_squared_error(y_test_pandas, y_pred_lr_pandas))
print("R2 Score:", r2_score(y_test_pandas, y_pred_lr_pandas))

print("\nLinear Regression - Polars")
print("MAE:", mean_absolute_error(y_test_polars, y_pred_lr_polars))
print("MSE:", mean_squared_error(y_test_polars, y_pred_lr_polars))
print("R2 Score:", r2_score(y_test_polars, y_pred_lr_polars))


Linear Regression - Pandas
MAE: 57.89402660797423
MSE: 16435.91415255836
R2 Score: -0.008729990161088885

Linear Regression - Polars
MAE: 57.51241224271416
MSE: 16498.701085393113
R2 Score: -0.012899836647937812


In [37]:
from sklearn.ensemble import RandomForestRegressor

rf_model_pandas = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_polars = RandomForestRegressor(n_estimators=100, random_state=42)


In [38]:
rf_model_pandas.fit(X_train_pandas_scaled, y_train_pandas.to_numpy().ravel())
rf_model_polars.fit(X_train_polars_scaled, y_train_polars.to_numpy().ravel())

RandomForestRegressor(random_state=42)

In [39]:
y_pred_rf_pandas = rf_model_pandas.predict(X_test_pandas_scaled)
y_pred_rf_polars = rf_model_polars.predict(X_test_polars_scaled)

In [40]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("Random Forest - Pandas")
print("MAE:", mean_absolute_error(y_test_pandas, y_pred_rf_pandas))
print("MSE:", mean_squared_error(y_test_pandas, y_pred_rf_pandas))
print("R2 Score:", r2_score(y_test_pandas, y_pred_rf_pandas))

print("\nRandom Forest - Polars")
print("MAE:", mean_absolute_error(y_test_polars, y_pred_rf_polars))
print("MSE:", mean_squared_error(y_test_polars, y_pred_rf_polars))
print("R2 Score:", r2_score(y_test_polars, y_pred_rf_polars))


Random Forest - Pandas
MAE: 62.61157898622061
MSE: 17011.723205118833
R2 Score: -0.04406942151444526

Random Forest - Polars
MAE: 60.97696829508594
MSE: 18231.334079182452
R2 Score: -0.11927085745113297
